# Sharpen
Try to sharpen images

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/MyDrive/M_DV_V2022

Mounted at /content/gdrive
/content/gdrive/MyDrive/M_DV_V2022


### Imports

In [2]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Input, Dense, Conv2D, Flatten, \
                         UpSampling2D, Conv2DTranspose, MaxPooling2D

### Load data

In [5]:
X = np.load("X_data.npy")
y = np.load("y_data.npy")

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.2, 
                                                    random_state=42)

### Make model

In [6]:
model = Sequential()

model.add(Input(shape=(60,60,1)))
model.add(Conv2D(32, kernel_size=3, activation='relu',
                     padding='valid'))
model.add(MaxPooling2D(2))
model.add(Conv2D(32, kernel_size=3, activation='relu',
                padding='same'))
model.add(UpSampling2D(5))
model.add(Conv2D(32, kernel_size=3, activation='relu',
                 padding='valid'))
model.add(MaxPooling2D(2))
model.add(UpSampling2D(2))

model.add(Conv2D(1, kernel_size=3, activation='linear',
                 padding='valid'))

#model.add(Conv2DTranspose(32, kernel_size=3, padding="same"))
#model.add(UpSampling2D(3))

model.summary()
model.compile(optimizer='adam',
              loss='mse')

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 58, 58, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 29, 29, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 up_sampling2d (UpSampling2D  (None, 145, 145, 32)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 143, 143, 32)      9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 71, 71, 32)       0

## Train model

In [ ]:
history = model.fit(X_train,y_train,
                    validation_data=(X_test,y_test),
                    epochs=200, batch_size=10)

Epoch 1/200
8/8 [==============================] - 7s 775ms/step - loss: 3337.8503 - val_loss: 3064.6707
Epoch 2/200
8/8 [==============================] - 6s 750ms/step - loss: 2853.8716 - val_loss: 2942.7241
Epoch 3/200
8/8 [==============================] - 6s 742ms/step - loss: 2765.1245 - val_loss: 2897.6831
Epoch 4/200
8/8 [==============================] - 6s 747ms/step - loss: 2697.0317 - val_loss: 2853.2749
Epoch 5/200
8/8 [==============================] - 6s 752ms/step - loss: 2637.0464 - val_loss: 2811.6304
Epoch 6/200
8/8 [==============================] - 6s 747ms/step - loss: 2601.8049 - val_loss: 2789.8225
Epoch 7/200
6/8 [=====================>........] - ETA: 1s - loss: 2611.8115